In [1]:
from math import sqrt
import numpy as np
from numpy.typing import NDArray
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint

from qoptcraft.basis import get_photon_basis
from qoptcraft.state import PureState, MixedState, Fock
from qoptcraft.invariant import can_transition, photon_invariant

In [2]:
def real_to_complex(z: NDArray) -> NDArray:
    return z[: len(z) // 2] + 1j * z[len(z) // 2 :]


def complex_to_real(z: NDArray):
    return np.concatenate((np.real(z), np.imag(z)))

In [3]:
modes = 6
photons = 4
photon_basis = get_photon_basis(modes, photons)
fock_list = [fock for fock in photon_basis if fock[-1] != 1 or fock[-2] != 1]


def invariant(x: NDArray) -> float:
    """Generate a constraint to make the invariant equal to zero.

    Args:
        x (NDArray): amplitudes of our quantum states.

    Returns:
        float: sum of square differences between in and out invariants
            for all the input and output states.
    """
    x = real_to_complex(x)
    fock_in_list = [(1, 0, 1, 0, 1, 1), (1, 0, 0, 1, 1, 1), (0, 1, 1, 0, 1, 1), (0, 1, 0, 1, 1, 1)]
    # x[0] = x[0] * 2
    x = x / np.sqrt(np.sum(np.abs(x) ** 2))
    AMP = x[0]
    # AMP = np.sqrt(2 / 27)

    rng = np.random.default_rng(15)

    invariant_constraint = 0
    for _ in range(20):
        coefs_in = np.sqrt(rng.uniform(0, 1, 4)) * np.exp(1.0j * rng.uniform(0, 2 * np.pi, 4))
        coefs_in = coefs_in / np.sum(np.abs(coefs_in) ** 2)
        state_in = PureState(fock_in_list, coefs_in)
        inv_in = photon_invariant(state_in, method="reduced")

        coefs_out = AMP * np.diag([1, 1, 1, -1]) @ coefs_in
        # state_out = PureState([*fock_in_list, *fock_list], np.append(coefs_out, x[1:]))
        state_out = PureState([*fock_in_list, *fock_list], np.append(coefs_out, x[1:]))
        inv_out = photon_invariant(state_out, method="reduced")
        invariant_constraint += (inv_in - inv_out) ** 2

    print(f"prob = {np.abs(AMP)**2}")
    print(f"Invariant = {invariant_constraint}\n")

    return invariant_constraint

In [4]:
x_init = np.ones(2 + 2 * len(fock_list))
method = "SLSQP"
constr_prob = {"type": "eq", "fun": lambda x: 1 - x @ x}
(res := minimize(invariant, x_init, method=method, constraints=constr_prob))

prob = 0.008547008547008546
Invariant = 374.2855864249399

prob = 0.008547008673280353
Invariant = 374.2855863093881

prob = 0.008547008545919998
Invariant = 374.28558634418533

prob = 0.008547008545919998
Invariant = 374.2855864162682

prob = 0.008547008545919998
Invariant = 374.2855864161875

prob = 0.008547008545919998
Invariant = 374.28558641624454

prob = 0.008547008545919998
Invariant = 374.2855863814524

prob = 0.008547008545919998
Invariant = 374.2855863814524

prob = 0.008547008545919998
Invariant = 374.2855864275867

prob = 0.008547008545919998
Invariant = 374.2855864854307

prob = 0.008547008545919998
Invariant = 374.2855864854878

prob = 0.008547008545919998
Invariant = 374.2855864417119

prob = 0.008547008545919998
Invariant = 374.285586441712

prob = 0.008547008545919998
Invariant = 374.2855864274695

prob = 0.008547008545919998
Invariant = 374.28558648540866

prob = 0.008547008545919998
Invariant = 374.28558644072984

prob = 0.008547008545919998
Invariant = 374.285586440

 message: Iteration limit reached
 success: False
  status: 9
     fun: 0.02024464071332439
       x: [ 3.282e-01  4.489e-03 ... -1.300e-02 -1.676e-03]
     nit: 100
     jac: [-3.459e-03  2.269e-03 ... -7.406e-04  1.560e-04]
    nfev: 23569
    njev: 100

In [8]:
np.sum(np.abs(res.x[:2]) ** 2)

0.43743398975650843

In [9]:
x_init = np.ones(2 + 2 * len(fock_list))
method = "L-BFGS-B"
(res := minimize(invariant, x_init, method=method))

prob = 0.008547008547008546
Invariant = 385.690915062575

prob = 0.008547008631748117
Invariant = 385.69091496214105

prob = 0.00854700854627803
Invariant = 385.690915007886

prob = 0.00854700854627803
Invariant = 385.6909150568691

prob = 0.00854700854627803
Invariant = 385.69091505682013

prob = 0.00854700854627803
Invariant = 385.69091505689084

prob = 0.00854700854627803
Invariant = 385.69091503323386

prob = 0.00854700854627803
Invariant = 385.6909150332339

prob = 0.00854700854627803
Invariant = 385.6909150645552

prob = 0.00854700854627803
Invariant = 385.69091510387244

prob = 0.00854700854627803
Invariant = 385.6909151039432

prob = 0.00854700854627803
Invariant = 385.6909150741702

prob = 0.00854700854627803
Invariant = 385.6909150741703

prob = 0.00854700854627803
Invariant = 385.6909150644808

prob = 0.00854700854627803
Invariant = 385.6909151038947

prob = 0.00854700854627803
Invariant = 385.69091507389106

prob = 0.00854700854627803
Invariant = 385.69091507389095

prob = 

  message: STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
  success: False
   status: 1
      fun: 0.06546256376410518
        x: [ 1.186e+01 -4.665e-01 ... -9.034e-01  9.823e-02]
      nit: 57
      jac: [-3.546e-06 -3.491e-05 ...  3.681e-05  7.824e-06]
     nfev: 15040
     njev: 64
 hess_inv: <234x234 LbfgsInvHessProduct with dtype=float64>

In [10]:
np.sum(np.abs(res.x[:2]) ** 2)

140.92502411091266

## Solve unitary equation with sympy

First we create a basis of the unitary matrices